Here's how you can load file from your google Drive. (the path might change)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import gzip

with gzip.open('/content/gdrive/My Drive/Columbia/Personalization/Final Project/Beeradvocate.txt.gz', 'r') as f:
  rb_file = f.readlines()

In [3]:
#Use this if your file is in txt format

with open('your path', 'r') as f:
  rb_file = f.readlines()

FileNotFoundError: ignored

In [0]:
# python2
import pandas as pd
import numpy as np
import os

In [0]:
data = []
row_out = []

for i in rb_file:
    row = i.decode('utf-8', errors = 'ignore')
    #print(row)
    if row == '\n':
      data.append(row_out)
      row_out = []
      continue
    row = row.split(":", 1)[1].rstrip()
    row_out.append(row)

        
        

In [6]:
data = pd.DataFrame(data)

data.columns = ['beer_name', 'beer_beerId', 'beer_brewer', 'beer_ABV', 'beer_style', 
                'review_appearance', 'review_aroma', 'review_palate', 'review_taste', 
                'review_overall', 'review_time', 'review_profileName', 'review_text']

# keep 3 columns: user name, beer name, overall score
data2 = data[['beer_name', 'review_profileName', 'review_overall','review_time']]
#print(data2.shape)

# remove NA
data2 = data2[pd.notnull(data2.beer_name)]
data2 = data2[pd.notnull(data2.review_profileName)]
data2 = data2[pd.notnull(data2.review_overall)]
print(data2.shape)
#data2.head(10)

(1586614, 4)


In [7]:
# keep top 2000/33382 frequent users
user = data2.review_profileName.value_counts()
user_list = user.keys()[:2000].tolist()

# keep top 200/56855 most reviewed beer
beer = data2.beer_name.value_counts()[:200]
beer_list = beer.keys()[:200].tolist()

# keep (beer&user) pair in (user_list) and (beer_list)
subdata = data2[data2.beer_name.isin(beer_list)]
subdata = subdata[subdata.review_profileName.isin(user_list)]

# sort by user names
subdata = subdata.sort_values(by=['review_profileName','beer_name','review_time'])

print(subdata.shape)
subdata.head(10)

(177323, 4)


,beer_name,review_profileName,review_overall,review_time
203255,1554 Enlightened Black Ale,,5,1230572555
1171411,60 Minute IPA,,5,1092634453
1170196,60 Minute IPA,,4.5,1231377089
1175028,90 Minute IPA,,5,1120085235
179764,AleSmith Speedway Stout,,4.5,1104186971
1320696,Anchor Porter,,5,1085582845
1322585,Anchor Steam Beer,,3.5,1161395034
1190684,ApriHop,,4,1238969849
869290,Arrogant Bastard Ale,,4.5,1082163606
868115,Arrogant Bastard Ale,,4.5,1215045531


Only keeping latest reviews by user in case they rated the same beer multipe times

In [0]:
#subdata.groupby(['beer_name', 'review_profileName'])['review_time'].max()

data_clean = subdata.sort_values(by='review_time')

data_clean.drop_duplicates(['beer_name', 'review_profileName'], keep = 'last',
                          inplace = True)


In [9]:
data_clean.head(10)
#data_clean.shape

,beer_name,review_profileName,review_overall,review_time
282223,Samuel Adams Boston Lager,Jason,4,1000083887
292025,Samuel Adams Octoberfest,AaronRed,4,1000135216
518392,Samuel Smith's Oatmeal Stout,Jason,4,1000336190
518391,Samuel Smith's Oatmeal Stout,guinness33,5,1000464943
1440424,Black Butte Porter,Tim,5,1000674768
1339361,Alpha King Pale Ale,John,5,1000733852
1148316,Fuller's ESB,John,5,1000804529
1541504,Golden Monkey,John,5,1000809985
344591,Orval Trappist Ale,AaronRed,5,1000832810
275832,Hoegaarden Original White Ale,AaronRed,4,1000833130


**Splitting the data into training and testing**

In [0]:
data_clean['review_overall'] = data_clean['review_overall'].astype(float)

data_train = data_clean.sample(frac = 0.7)
data_test = data_clean.drop(data_train.index)

Calculate the average rating for each item

In [0]:
mean_rating = data_train.groupby(['beer_name'], as_index = False, sort = False)['review_overall'].mean().rename(columns = {'review_overall': 'item_mean'})

data_train = pd.merge(data_train, mean_rating, on = 'beer_name', how = 'left')
data_train['adjusted_rating'] = data_train['review_overall'] - data_train['item_mean']

In [0]:
data_train.head()

,beer_name,review_profileName,review_overall,review_time,item_mean,adjusted_rating
0,Samuel Smith's Nut Brown Ale,WhiteOak,4.0,1272731933,4.123092,-0.123092
1,Tripel Karmeliet,sabrills70,4.0,1167525395,4.207447,-0.207447
2,Stone Ruination IPA,Morris729,5.0,1078039289,4.153217,0.846783
3,Samuel Smith's Nut Brown Ale,BierFan,4.0,1228430192,4.123092,-0.123092
4,Sierra Nevada Pale Ale,BigPlay1824,4.0,1322003081,4.257381,-0.257381


**Build the similarity matrix**

In [0]:
def item_matrix(train_sample):
  #find distinct beers
  beers = np.unique(train_sample['beer_name'])
  beer_pairs = pd.DataFrame(columns = ['user', 'beer1', 'beer2', 'scaled_r1', 'scaled_r2'])

  for beer1 in beers:
    #determine users who consumed the beer
    users = train_sample.loc[train_sample['beer_name'] == beer1]['review_profileName']    
    for user in users:
      #find rating for first beer
      beer1_rating = train_sample.loc[train_sample['beer_name'] == beer1]['adjusted_rating']
      #find other beer ratings by same users, excluding beer1 rating
      user_ratings = train_sample.loc[(train_sample['review_profileName'] == user) & (train_sample['beer_name'] != beer1)]
      for index, beer2 in user_ratings.iterrows():
        #print (beer2)
        beer2_name = beer2['beer_name']
        beer2_rating = beer2['adjusted_rating']
        beer_pairs.append([user, beer1, beer2_name, beer1_rating, beer2_rating])

  return beer_pairs
      

In [0]:
beer_pairs = item_matrix(data_train)
#beers = np.unique(data_train['beer_name'])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexes/api.py:87: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


In [0]:
#data_train.iloc[0]['beer_name']
for i in data_train.loc[data_train['beer_name'] == " Samuel Smith's Nut Brown Ale"]['review_profileName']:
  print(i)

 WhiteOak
 BierFan
 mintjellie
 albern
 zoso1967
 erosier
 northyorksammy
 Phatz
 DESTRO
 jwc215
 flipper2gv
 Strix
 cubedbee
 Taelec
 mynie
 Jason
 claspada
 russpowell
 jminsc76
 ludachris
 MJR
 Gueuzedude
 cokes
 jujubeast6000
 KoG
 FanofHefe
 seanyfo
 Rootdog316
 david131
 baos
 Stimack
 nasty15108
 Viggo
 WesWes
 hardy008
 Lupe
 Slynger
 TLove
 TheSarge
 Derek
 fitzy84
 srandycarter
 demcorhip
 MbpBugeye
 MMansfield
 HalfFull
 clvand0
 feloniousmonk
 Thrasher
 adam42381
 longpondbeerman
 Kinsman
 ZenAgnostic
 BoitSansSoif
 StevieW
 jeranbrews
 necoadam
 Crosling
 roadhouse
 CampusCrew
 morbiddrumer
 EnojysAnyBeer
 BeerSox
 Frozensoul327
 mschofield
 taigor
 froghop
 naterock
 sacfly
 Jesstyr
 BEERchitect
 JOaikido
 bundy462
 zhanson88
 Tupperwolf
 EvilKeith
 Ahhdball
 biggmike
 SkeeterHawk
 packetknife
 joemcgrath27
 Graham5161986
 jwhawkins81
 largadeer
 Brunite
 stephendr
 matty
 dacrza1
 HopsAreOverrated
 steveh
 geexploitation
 Fatehunter
 ViveLaChouffe
 Kendo
 Pegasus
 nrpell